In [5]:
from typing import Optional
from pydantic import BaseModel
from pyfuseki import FusekiUpdate

In [13]:
class Element(BaseModel):
    elementType: str
    value: str

class Variant(BaseModel):
    variantType: str
    name: Element
    date: Optional[Element]

In [34]:
obj = {
    "variantType": "PersonalName",
    "name": {
            "elementType": "FullNameElement",
            "value": "Сарамаго, Жозе"
        }
}
request = Variant(**obj)
request

Variant(variantType='PersonalName', name=Element(elementType='FullNameElement', value='Сарамаго, Жозе'), date=None)

In [35]:
from api.src.function.authorities.agents.hasVariant import DeleteVariant


response = DeleteVariant("https://bibliokeia.com/authorities/PersonalName/bkau-1", request)
response

{'statusCode': 200, 'message': 'Update succeeded'}

In [20]:
upAuthorities = FusekiUpdate('http://localhost:3030', 'authorities')

In [31]:
variant = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        WITH <https://bibliokeia.com/authorities/PersonalName/bkau-1>

        DELETE {{
<https://bibliokeia.com/authorities/PersonalName/bkau-1> madsrdf:hasVariant ?hasVariant .
    ?hasVariant rdf:type madsrdf:Variant, madsrdf:{request.variantType} .
    ?hasVariant madsrdf:variantLabel ?variantLabel .
    ?hasVariant madsrdf:elementList ?elementList .
    ?elementList rdf:first ?first .
    ?first madsrdf:elementValue "{request.name.value}" .
    ?first rdf:type madsrdf:{request.name.elementType} .   
    ?elementList rdf:rest rdf:nil .  
     {  '?elementList rdf:rest ?rest .' if request.date else '' } 
     {  '?rest rdf:rest rdf:nil . ' if request.date else '' } 
     {  '?rest rdf:first ?date . ' if request.date else '' } 
     {  '?date rdf:type madsrdf:DateNameElement . ' if request.date else '' } 
     {  f'?date madsrdf:elementValue "{request.date.value}"' if request.date else '' }     
        }}

    WHERE {{ 
       <https://bibliokeia.com/authorities/PersonalName/bkau-1> madsrdf:hasVariant ?hasVariant .
    ?hasVariant rdf:type madsrdf:Variant, madsrdf:{request.variantType} .
    ?hasVariant madsrdf:variantLabel ?variantLabel .
    ?hasVariant madsrdf:elementList ?elementList .
    ?elementList rdf:first ?first .
    ?first madsrdf:elementValue "{request.name.value}" .
    ?first rdf:type madsrdf:{request.name.elementType} . 
    ?elementList rdf:rest rdf:nil .  
     {  '?elementList rdf:rest ?rest .' if request.date else '' } 
     {  '?rest rdf:rest rdf:nil . ' if request.date else '' } 
     {  '?rest rdf:first ?date . ' if request.date else '' } 
     {  '?date rdf:type madsrdf:DateNameElement . ' if request.date else '' } 
     {  f'?date madsrdf:elementValue "{request.date.value}"' if request.date else '' }  
        }}"""

r = upAuthorities.run_sparql(variant)
r.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [8]:
delete = """PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
DELETE DATA
{ GRAPH <https://bibliokeia.com/authorities/PersonalName/bkau-1> { 
    <https://bibliokeia.com/authorities/PersonalName/bkau-1> madsrdf:hasVariant [
    rdf:type madsrdf:Variant , madsrdf:PersonalName ;
    madsrdf:elementList (
        [ rdf:type  madsrdf:FullNameElement ;
          madsrdf:elementValue  "FLAVIO GUSTAVO" ]
        [ rdf:type madsrdf:DateNameElement ;
          madsrdf:elementValue  "1821-1880" ] ) ;
  madsrdf:variantLabel  "Flaubert, Gustaw,, 1821-1880" ]
      } }"""

In [9]:
upAcervo = FusekiUpdate('http://localhost:3030', 'authorities')

r = upAcervo.run_sparql(delete)
r.convert()

QueryBadFormed: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Line 4, column 81: Blank nodes not allowed in DELETE templates\n'

In [ ]:
madsrdf:hasVariant          [ rdf:type              madsrdf:Variant , madsrdf:PersonalName ;
                                      madsrdf:elementList   ( [ rdf:type              madsrdf:FullNameElement ;
                                                                madsrdf:elementValue  "FLAVIO GUSTAVO"
                                                              ]
                                                              [ rdf:type              madsrdf:DateNameElement ;
                                                                madsrdf:elementValue  "1821-1880"
                                                              ]
                                                            ) ;
                                      madsrdf:variantLabel  "Flaubert, Gustaw,, 1821-1880"
                                    ] ;